In [3]:
__author__ = "Yicheng Li"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import random
from sklearn import preprocessing
import tensorflow as tf

dir_path = 'CS341-repo/Data/'
df = pd.read_pickle(dir_path+'df_hourly_poloniex.pickle')
df = df.dropna()

df.head()

/home/yicheng/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,USDT_BTC_high,USDT_BTC_low,USDT_BTC_close,USDT_BTC_open,USDT_BTC_volume,USDT_BTC_quoteVolume,USDT_BTC_weighted_mean,USDT_BTC_pctChange,USDT_ETH_high,USDT_ETH_low,...,BTC_LTC_weighted_mean,BTC_LTC_pctChange,BTC_XRP_high,BTC_XRP_low,BTC_XRP_close,BTC_XRP_open,BTC_XRP_volume,BTC_XRP_quoteVolume,BTC_XRP_weighted_mean,BTC_XRP_pctChange
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 12:00:00,432.5000,432.50,432.500000,432.50000,40.041239,0.092581,432.500000,2.220446e-16,0.959136,0.959136,...,0.008063,-0.002293,0.000014,0.000014,0.000014,0.000014,0.033605,2408.822942,0.000014,-0.002859
2016-01-02 13:00:00,432.5000,432.50,432.500000,432.50000,0.000000,0.000000,432.986941,1.125876e-03,0.959136,0.959136,...,0.008060,-0.000333,0.000014,0.000014,0.000014,0.000014,0.000000,0.000000,0.000014,0.004704
2016-01-02 14:00:00,437.3635,432.48,433.336667,433.52799,359.269753,0.828819,433.473883,1.124610e-03,0.959136,0.957000,...,0.008073,0.001623,0.000014,0.000014,0.000014,0.000014,1.141981,81071.098773,0.000014,0.004682
2016-01-02 15:00:00,432.4800,432.48,432.480000,432.48000,60.859598,0.140722,432.480000,-2.292832e-03,0.957000,0.957000,...,0.008089,0.002002,0.000014,0.000014,0.000014,0.000014,2.120423,150622.792769,0.000014,-0.000492
2016-01-02 16:00:00,432.4800,432.48,432.480000,432.48000,0.000000,0.000000,432.350000,-3.005919e-04,0.957000,0.957000,...,0.008079,-0.001224,0.000014,0.000014,0.000014,0.000014,0.491516,35178.793196,0.000014,-0.007526


In [72]:
scaler_features = preprocessing.MinMaxScaler(feature_range=(0.1, 1))

In [73]:
# function to create train, validation, test data given sequence length
def load_data(df, seq_len):
    # prepare one-hot labels
    labels = df['USDT_BTC_pctChange'].as_matrix().reshape([-1,1])
    labels = np.concatenate([(labels > 3e-3)*1, ((3e-3 > labels)&(labels > -3e-3))*1, (labels < -3e-3)*1],1)
    feature_set = [x for x in range(56)] #[0,1,2,3,4,6,7]
    
    data_raw = df.as_matrix() # convert to numpy array
    # fit scaler
    data_raw = scaler_features.fit_transform(data_raw[:, feature_set])
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len + 1): 
        data.append(data_raw[index: index + seq_len, :])
    
    data = np.array(data)
    
    n_train_valid_pairs = 3
    each_train_set_size_pct = 25
    each_valid_set_size_pct = 5
    
    each_train_set_size = round(each_train_set_size_pct/100*data.shape[0])
    each_valid_set_size = round(each_valid_set_size_pct/100*data.shape[0])
    
    x_train_sets = []
    y_train_sets = []
    x_valid_sets = []
    y_valid_sets = []
    used = 0
    
    for i in range(n_train_valid_pairs):
        x_train_sets.append(data[used : used + each_train_set_size,:-1,:]) # cannot see last day, which we aim to predict
        y_train_sets.append(labels[used + seq_len-1 : used + each_train_set_size + seq_len-1, :])
        used += each_train_set_size
    
        x_valid_sets.append(data[used : used + each_valid_set_size,:-1,:])
        y_valid_sets.append(labels[used + seq_len-1 : used + each_valid_set_size + seq_len-1, :])
        used += each_valid_set_size
    
    x_test = data[used : , :-1, :]
    y_test = labels[seq_len-1 + used : , :]
    
    x_train = np.concatenate(x_train_sets, axis=0)
    y_train = np.concatenate(y_train_sets, axis=0)
    x_valid = np.concatenate(x_valid_sets, axis=0)
    y_valid = np.concatenate(y_valid_sets, axis=0)
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

In [74]:
# create train, test data
seq_len = 10 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df, seq_len)
# y_train = y_train.reshape([-1,1])
# y_valid = y_valid.reshape([-1,1])
# y_test = y_test.reshape([-1,1])
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (14982, 9, 56)
y_train.shape =  (14982, 3)
x_valid.shape =  (2997, 9, 56)
y_valid.shape =  (2997, 3)
x_test.shape =  (1996, 9, 56)
y_test.shape =  (1996, 3)


In [7]:
def f1_score_single(y_true, y_pred):
    TP = y_true.dot(y_pred) # zero or one
    FP = np.sum(y_pred > y_true) # sum over all k classes, zero or one
    FN = np.sum(y_pred < y_true) # sum over all k classes, zero or one
    
    if TP == 0: return 0.
    p = 1. * TP / (TP + FP)
    r = 1. * TP / (TP + FN)
    return 2 * p * r / (p + r)
    
def F1(y_true, y_pred):
    return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])

## baseline

In [8]:
y_pred = np.roll(y_valid,1, axis=0)
print('baseline dev_F1=',F1(y_valid[1:], y_pred[1:]))
y_pred = np.roll(y_test,1, axis=0)
y_pred[0] = y_valid[-1] # be careful here
print('baseline test_F1=',F1(y_test, y_pred))

baseline dev_F1= 0.49345857094934586
baseline test_F1= 0.3708542713567839


In [75]:
index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

x_1000_train, y_1000_train = get_next_batch(1000) # special batch of 1000 records in training set

In [111]:
class LSTM_Model(object):
    def __init__(self, seq_len):
        # parameters
        self.n_steps = seq_len-1 
        self.n_inputs = x_train.shape[-1]
        self.n_neurons = 100  # cell.state_size
        self.n_bins = 3 # be careful if you want to change this
        self.n_layers = 2
        self.batch_size = 100
        self.n_epochs = 0 # 0 means to train indefinitely
        self.train_set_size = x_train.shape[0]
        self.test_set_size = x_test.shape[0]
        self.keep_prob = 0.9
        self.max_gradient_norm = 5
        with tf.variable_scope("LSTM_Model", initializer=tf.contrib.layers.xavier_initializer()):
            self.X = tf.placeholder(tf.float32, [None, self.n_steps, self.n_inputs])
            self.y = tf.placeholder(tf.float32, [None, self.n_bins])

            layers = [tf.contrib.rnn.LSTMCell(num_units=self.n_neurons, \
                                              initializer=tf.contrib.layers.xavier_initializer(), \
                                              activation=tf.nn.elu)
                     for layer in range(self.n_layers)]
            
#             layers = [tf.contrib.rnn.BasicRNNCell(num_units=self.n_neurons, \
#                                               activation=tf.nn.elu)
#                      for layer in range(self.n_layers)]
            
            multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

            outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, self.X, dtype=tf.float32)
            outputs = tf.nn.dropout(outputs, self.keep_prob)
            # 'outputs' is a tensor of shape [batch_size, n_steps, n_neurons(cell.state_size)]
        
#             directly output
#             logits = outputs[:,self.n_steps-1,:] # keep only last output of sequence
            
            # ======== attn layer ===================
#             sim_mat = tf.matmul(outputs, tf.transpose(outputs, perm=[0,2,1]))
#             attn_dist = tf.nn.softmax(sim_mat, 2)
#             # (batchsize, n_steps, n_steps)
#             attn_outputs = tf.matmul(attn_dist, outputs)
#             # (batchsize, n_steps, n_bins)
            # ========================================
            
            stacked_outputs = tf.reshape(outputs, [-1, self.n_neurons]) 
            stacked_outputs = tf.layers.dense(stacked_outputs, self.n_bins)
            final_outputs = tf.reshape(stacked_outputs, [-1, self.n_steps, self.n_bins])
            
            self.final_logits = final_outputs[:, -1, :] # last timestep
            # (batchsize, n_bins), this is logits, not probs!!
            self.indices = tf.argmax(self.final_logits, axis=-1) # (batchsize, 1)
            self.preds = tf.one_hot(self.indices, depth=self.n_bins)
            
            self.each_loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.final_logits, labels=self.y)
            self.loss = tf.reduce_mean(self.each_loss) 

            params = tf.trainable_variables()
            gradients = tf.gradients(self.loss, params)
            self.gradient_norm = tf.global_norm(gradients)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
            clipped_norm = tf.global_norm(clipped_gradients)
            self.param_norm = tf.global_norm(params)
            self.learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
            optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate_placeholder) 
            # training_op = optimizer.minimize(loss)
            self.training_op = optimizer.apply_gradients(zip(clipped_gradients, params))

            # initialize parameters
#             sess = tf.Session()
            self.global_step = tf.Variable(0, name='global_step', trainable=False)
            self.saver = tf.train.Saver(max_to_keep=2)
    
    def train(self, session, experiment_name):
        ckpt = tf.train.get_checkpoint_state(experiment_name)
        v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
        if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
            self.saver.restore(sess, ckpt.model_checkpoint_path)
            iteration = self.global_step.eval(session=sess) # get last global_step
            print("Start from iteration:", iteration)
            lr = 1e-3
        else:
            print('There is not saved parameters. Creating model with fresh parameters.')
            sess.run(tf.global_variables_initializer())
            iteration = 0
            lr = 1e-2 # should fix this...

        old_loss = 1000
        
        while self.n_epochs == 0 or iteration*self.batch_size/self.train_set_size < self.n_epochs:
            iteration = iteration + 1
            x_batch, y_batch = get_next_batch(self.batch_size) # fetch the next training batch 

            # train on this batch
            sess.run(self.training_op, feed_dict={self.X: x_batch, self.y: y_batch, self.learning_rate_placeholder:lr})

            if iteration % 20 == 0:
                y_1000_train_pred, loss_val, param_norm_val, grad_norm_val = sess.run([self.preds, self.loss, self.param_norm, self.gradient_norm],\
                                                                        feed_dict={self.X: x_1000_train, self.y:y_1000_train, \
                                                                                  self.learning_rate_placeholder:lr})
                if loss_val > old_loss * 1.2:
                    lr /= 2
                old_loss = loss_val

                y_valid_pred = sess.run(self.preds, feed_dict={self.X: x_valid})
                print('%.2f epochs, iter %d: train_loss = %.9f, param_norm = %.3f, grad_norm = %.3f, train_F1/valid_F1 = %.6f/%.6f' \
                      %(iteration*self.batch_size/self.train_set_size, iteration, loss_val, param_norm_val, grad_norm_val, \
                        F1(y_1000_train, y_1000_train_pred), \
                        F1(y_valid, y_valid_pred)))
        #             print('%.2f epochs: loss train/valid = %.6f/%.6f'%(
        #                 iteration*batch_size/train_set_size, loss_train, loss_valid))
            if iteration % 100 == 0:
                self.global_step.assign(iteration).eval(session=sess) # set and update(eval) global_step with index, i
                save_path = self.saver.save(sess, "./"+experiment_name+"/model.ckpt", global_step=self.global_step)
                print('Saved parameters to %s' % save_path)

## run experiment

In [92]:
experiment_name="all_features_10_seq_100hidden_2layer"

In [112]:
tf.reset_default_graph()
lstm_model = LSTM_Model(seq_len=seq_len)
sess = tf.Session()

In [117]:
lstm_model.train(session=sess, experiment_name=experiment_name)
# IMPORTANT:
# when you think F1 is not going to improve anymore, wait another 10 epochs. 
# if you see any better iteration that has not appeared before, keep waiting.

INFO:tensorflow:Restoring parameters from all_features_10_seq_100hidden_2layer/model.ckpt-12700
Start from iteration: 12700
84.90 epochs, iter 12720: train_loss = 0.893858790, param_norm = 150.472, grad_norm = 0.085, train_F1/valid_F1 = 0.585000/0.523190
85.04 epochs, iter 12740: train_loss = 0.897749186, param_norm = 150.496, grad_norm = 0.088, train_F1/valid_F1 = 0.575000/0.518519
85.17 epochs, iter 12760: train_loss = 0.904023051, param_norm = 150.479, grad_norm = 0.073, train_F1/valid_F1 = 0.567000/0.518852
85.30 epochs, iter 12780: train_loss = 0.896291614, param_norm = 150.487, grad_norm = 0.072, train_F1/valid_F1 = 0.572000/0.535202
85.44 epochs, iter 12800: train_loss = 0.899986863, param_norm = 150.506, grad_norm = 0.083, train_F1/valid_F1 = 0.566000/0.523524
Saved parameters to ./all_features_10_seq_100hidden_2layer/model.ckpt-12800
85.57 epochs, iter 12820: train_loss = 0.896343887, param_norm = 150.542, grad_norm = 0.082, train_F1/valid_F1 = 0.575000/0.526527
85.70 epochs, 

92.38 epochs, iter 13840: train_loss = 0.899423361, param_norm = 151.113, grad_norm = 0.094, train_F1/valid_F1 = 0.573000/0.529530
92.51 epochs, iter 13860: train_loss = 0.907877386, param_norm = 151.127, grad_norm = 0.444, train_F1/valid_F1 = 0.581000/0.518185
92.64 epochs, iter 13880: train_loss = 0.893178105, param_norm = 151.133, grad_norm = 0.126, train_F1/valid_F1 = 0.580000/0.528195
92.78 epochs, iter 13900: train_loss = 0.896203220, param_norm = 151.129, grad_norm = 0.276, train_F1/valid_F1 = 0.573000/0.522856
Saved parameters to ./all_features_10_seq_100hidden_2layer/model.ckpt-13900
92.91 epochs, iter 13920: train_loss = 0.900009394, param_norm = 151.147, grad_norm = 0.096, train_F1/valid_F1 = 0.568000/0.519520
93.04 epochs, iter 13940: train_loss = 0.894051433, param_norm = 151.171, grad_norm = 0.123, train_F1/valid_F1 = 0.577000/0.519520
93.18 epochs, iter 13960: train_loss = 0.900637329, param_norm = 151.192, grad_norm = 0.332, train_F1/valid_F1 = 0.578000/0.526527
93.31 e

99.99 epochs, iter 14980: train_loss = 0.890790284, param_norm = 151.770, grad_norm = 0.106, train_F1/valid_F1 = 0.576000/0.521522
100.12 epochs, iter 15000: train_loss = 0.897155821, param_norm = 151.778, grad_norm = 0.777, train_F1/valid_F1 = 0.580000/0.521522
Saved parameters to ./all_features_10_seq_100hidden_2layer/model.ckpt-15000
100.25 epochs, iter 15020: train_loss = 0.898068488, param_norm = 151.780, grad_norm = 0.216, train_F1/valid_F1 = 0.572000/0.519520
100.39 epochs, iter 15040: train_loss = 0.890522659, param_norm = 151.798, grad_norm = 0.137, train_F1/valid_F1 = 0.572000/0.520854
100.52 epochs, iter 15060: train_loss = 0.886653364, param_norm = 151.808, grad_norm = 0.097, train_F1/valid_F1 = 0.585000/0.511845
100.65 epochs, iter 15080: train_loss = 0.890757263, param_norm = 151.814, grad_norm = 0.129, train_F1/valid_F1 = 0.578000/0.518519
100.79 epochs, iter 15100: train_loss = 0.890950918, param_norm = 151.861, grad_norm = 0.195, train_F1/valid_F1 = 0.581000/0.519520
S

107.46 epochs, iter 16100: train_loss = 0.882169902, param_norm = 152.577, grad_norm = 0.243, train_F1/valid_F1 = 0.583000/0.510177
Saved parameters to ./all_features_10_seq_100hidden_2layer/model.ckpt-16100
107.60 epochs, iter 16120: train_loss = 0.884269655, param_norm = 152.581, grad_norm = 0.234, train_F1/valid_F1 = 0.584000/0.531198
107.73 epochs, iter 16140: train_loss = 0.886056662, param_norm = 152.592, grad_norm = 0.326, train_F1/valid_F1 = 0.576000/0.521188
107.86 epochs, iter 16160: train_loss = 0.881670654, param_norm = 152.601, grad_norm = 0.489, train_F1/valid_F1 = 0.593000/0.536203
108.00 epochs, iter 16180: train_loss = 0.883905232, param_norm = 152.602, grad_norm = 0.168, train_F1/valid_F1 = 0.589000/0.509843
108.13 epochs, iter 16200: train_loss = 0.881518424, param_norm = 152.615, grad_norm = 0.143, train_F1/valid_F1 = 0.590000/0.505506
Saved parameters to ./all_features_10_seq_100hidden_2layer/model.ckpt-16200
108.26 epochs, iter 16220: train_loss = 0.889402747, par

KeyboardInterrupt: 

In [118]:
y_1000_train_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_1000_train})
print("train F1:",F1(y_1000_train, y_1000_train_pred))
y_valid_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_valid})
print("dev F1:",F1(y_valid, y_valid_pred))
y_test_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_test})
print("test F1:",F1(y_test, y_test_pred))

train F1: 0.584
dev F1: 0.5235235235235235
test F1: 0.3532064128256513
